In [4]:
import cloudy_input_builder as cib

In [5]:
x = cib.CloudyModel()
x.build_cleri_model(element_scale_factor_dict={'nitrogen':0.4,'oxygen':0.5})
x.make_cloudy_in_file()

In [6]:
metallicities = [0.1, 0.2, 0.3]
for i, z in enumerate(metallicities):
    model = cib.CloudyModel()
    model.build_cleri_model(gas_metallicity=z)
    model.make_cloudy_in_file()

In [7]:
stars = cib.CloudyModel()
stars.build_template_model_bpass()
stars.model

['table star "BPASSv2.2.1_imf135_300_burst_binary.ascii" 10000000.0 -1',
 'hden 2',
 'abundances gass10 no grains',
 'grains Orion',
 'metals and grains 1.0',
 'ionization parameter 0 vary',
 'grid -4 to -1 step 0.25',
 'iterate_to_convergence',
 'save overview ".ovr" last',
 'save continuum units angstroms ".con" last',
 'save linelist column emergent absolute last units angstroms ".elin" "LineList_HII_NJC.dat"',
 'save linelist column intrinsic absolute last units angstroms ".elin" "LineList_HII_NJC.dat"']